In [1]:
## To import the various libraries

from pymongo import MongoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sklearn
import datetime as dt
import random
import itertools
import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


In [2]:
## To import the data from MongoDB server as pandas DataFrame

client = MongoClient()
db = client.DataIntern
collection = db.AnomalyDetect
data = pd.DataFrame(list(collection.find()))
data

,_id,part_cycle_time,part_ID,program_number,part_count,quality,running_tool_number,tool_changed,spindle_load_%,spindle_speed,...,vibration_X,vibration_Z,vibration_spindle,noise,flow_rate,cutting_speed,depth_of_cut,diameter,tolerance,material
0,5f27c70e5fb3846d152450e0,2018-05-01 00:00:00,138,1000,1,NaN,10.0,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,NaN,0.386117,85,NaN,MILD_STEEL
1,5f27c70e5fb3846d152450e1,2018-05-01 00:00:01,138,1000,1,NaN,10.0,NaN,0.047170,0.377358,...,0.000943,0.000943,0.002830,0.004717,0,NaN,0.780012,85,NaN,MILD_STEEL
2,5f27c70e5fb3846d152450e2,2018-05-01 00:00:02,138,1000,1,NaN,10.0,NaN,0.094340,0.754717,...,0.001887,0.001887,0.005660,0.009434,0,NaN,0.990850,80,NaN,MILD_STEEL
3,5f27c70e5fb3846d152450e3,2018-05-01 00:00:03,138,1000,1,NaN,10.0,NaN,0.141509,1.132075,...,0.002830,0.002830,0.008491,0.014151,0,NaN,0.835825,80,NaN,MILD_STEEL
4,5f27c70e5fb3846d152450e4,2018-05-01 00:00:04,138,1000,1,NaN,10.0,NaN,0.188679,1.509434,...,0.003774,0.003774,0.011321,0.018868,0,NaN,0.921784,80,NaN,MILD_STEEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939279,5f27c71f5fb3846d1532a5ef,2018-05-01 00:13:06,150,1950,1000,NaN,10.0,NaN,11.111111,3000.000000,...,2.631579,3.666667,2.631579,99.411765,25,NaN,0.582694,85,NaN,MILD_STEEL
939280,5f27c71f5fb3846d1532a5f0,2018-05-01 00:13:07,150,1950,1000,NaN,10.0,NaN,8.333333,2000.000000,...,2.421053,3.500000,2.421053,100.000000,25,NaN,0.604042,80,NaN,MILD_STEEL
939281,5f27c71f5fb3846d1532a5f1,2018-05-01 00:13:08,150,1950,1000,NaN,10.0,NaN,5.555556,1000.000000,...,2.210526,3.333333,2.210526,10.000000,25,NaN,0.395931,85,NaN,MILD_STEEL
939282,5f27c71f5fb3846d1532a5f2,2018-05-01 00:13:09,150,1950,1000,NaN,10.0,NaN,2.777778,1000.000000,...,2.000000,3.166667,2.000000,5.000000,20,NaN,0.748624,80,NaN,MILD_STEEL


In [3]:
## To remove the unneccessary columns
print(data['_id'].nunique())
data=data.iloc[:,1:]

939284


In [4]:
## To store a copy of the dataframe

df=data

In [5]:
## To extract only the time part from a particular column storing datetime values

df['part_cycle_time']=pd.to_datetime(df['part_cycle_time']).dt.time

In [6]:
df['quality1']=df['quality']
df.drop("quality",axis=1,inplace=True)
df.rename(columns={"quality1":"quality"},inplace=True)

In [7]:
## To correct the values of certain columns

np.random.seed(1)
totalItem=1000
randomNum1=np.random.randint(60,1800,totalItem)
randomList=list(randomNum1)

# to generate 1000 evenly spaced numbers between 100 and 1100
ranList1=np.linspace(1,totalItem,totalItem,dtype="int")
ranList2=np.random.randint(1,200,totalItem)
#print(ranList2)


# to make the part_count and part_ID column 

lis=[]
lis1=[]
for i in range(len(randomList)):
    lis.append(np.repeat(ranList1[i],randomList[i]))
    lis1.append(np.repeat(ranList2[i],randomList[i]))
df['part_count']=list(itertools.chain.from_iterable(lis))
df['part_ID']=list(itertools.chain.from_iterable(lis1))   
df.tail(20)


,part_cycle_time,part_ID,program_number,part_count,running_tool_number,tool_changed,spindle_load_%,spindle_speed,current,spindle_temp,...,vibration_Z,vibration_spindle,noise,flow_rate,cutting_speed,depth_of_cut,diameter,tolerance,material,quality
939264,00:12:51,150,1950,1000,10.0,NaN,50.000000,9555.000000,30.000000,53.684211,...,3.000000,5.789474,90.588235,20,NaN,0.054040,85,NaN,MILD_STEEL,NaN
939265,00:12:52,150,1950,1000,10.0,NaN,50.000000,9686.666667,30.000000,52.368421,...,6.000000,5.578947,91.176471,20,NaN,0.377826,80,NaN,MILD_STEEL,NaN
939266,00:12:53,150,1950,1000,10.0,NaN,47.222222,9818.333333,28.333333,51.052632,...,5.833333,5.368421,91.764706,25,NaN,0.395736,85,NaN,MILD_STEEL,NaN
939267,00:12:54,150,1950,1000,10.0,NaN,44.444444,9950.000000,26.666667,49.736842,...,5.666667,5.157895,92.352941,25,NaN,0.062095,85,NaN,MILD_STEEL,NaN
939268,00:12:55,150,1950,1000,10.0,NaN,41.666667,10000.000000,25.000000,48.421053,...,5.500000,4.947368,92.941176,25,213.0,0.632733,85,NaN,MILD_STEEL,NaN
939269,00:12:56,150,1950,1000,10.0,NaN,38.888889,10000.000000,23.333333,47.105263,...,5.333333,4.736842,93.529412,25,51.0,0.632733,80,NaN,MILD_STEEL,NaN
939270,00:12:57,150,1950,1000,10.0,NaN,36.111111,10000.000000,21.666667,45.789474,...,5.166667,4.526316,94.117647,25,269.0,0.632733,80,NaN,MILD_STEEL,NaN
939271,00:12:58,150,1950,1000,10.0,NaN,33.333333,10000.000000,20.000000,44.473684,...,5.000000,4.315789,94.705882,25,207.0,0.632733,85,NaN,MILD_STEEL,NaN
939272,00:12:59,150,1950,1000,10.0,NaN,30.555556,10000.000000,18.333333,43.157895,...,4.833333,4.105263,95.294118,25,99.0,0.632733,85,NaN,MILD_STEEL,NaN
939273,00:13:00,150,1950,1000,10.0,NaN,27.777778,10000.000000,16.666667,41.842105,...,4.666667,3.894737,95.882353,25,200.0,0.632733,80,NaN,MILD_STEEL,NaN


In [8]:
## To get the information about the various columns in the dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939284 entries, 0 to 939283
Data columns (total 22 columns):
part_cycle_time        939284 non-null object
part_ID                939284 non-null int64
program_number         939284 non-null int64
part_count             939284 non-null int64
running_tool_number    937240 non-null float64
tool_changed           2044 non-null float64
spindle_load_%         939284 non-null float64
spindle_speed          939284 non-null float64
current                939284 non-null float64
spindle_temp           939284 non-null float64
feed                   939284 non-null float64
vibration_X            939284 non-null float64
vibration_Z            939284 non-null float64
vibration_spindle      939284 non-null float64
noise                  939284 non-null float64
flow_rate              939284 non-null int64
cutting_speed          28991 non-null float64
depth_of_cut           939284 non-null float64
diameter               939284 non-null int64
tolerance 

In [9]:
df.columns

Index(['part_cycle_time', 'part_ID', 'program_number', 'part_count',
       'running_tool_number', 'tool_changed', 'spindle_load_%',
       'spindle_speed', 'current', 'spindle_temp', 'feed', 'vibration_X',
       'vibration_Z', 'vibration_spindle', 'noise', 'flow_rate',
       'cutting_speed', 'depth_of_cut', 'diameter', 'tolerance', 'material',
       'quality'],
      dtype='object')

In [10]:
df.isnull().sum()

part_cycle_time             0
part_ID                     0
program_number              0
part_count                  0
running_tool_number      2044
tool_changed           937240
spindle_load_%              0
spindle_speed               0
current                     0
spindle_temp                0
feed                        0
vibration_X                 0
vibration_Z                 0
vibration_spindle           0
noise                       0
flow_rate                   0
cutting_speed          910293
depth_of_cut                0
diameter                    0
tolerance              938284
material                    0
quality                938284
dtype: int64

In [11]:
df.shape

(939284, 22)

In [12]:
df.describe().shape

(8, 19)

In [13]:
df.shape[1]-df.describe().shape[1]

3

In [14]:
df['part_ID'].value_counts()

181    13497
64     11582
152    11421
93     11236
183    10837
       ...  
103     1002
115      745
124      728
42       588
168      566
Name: part_ID, Length: 198, dtype: int64

In [15]:
df['part_ID'].max()

199

In [16]:
df.describe()

,part_ID,program_number,part_count,running_tool_number,tool_changed,spindle_load_%,spindle_speed,current,spindle_temp,feed,vibration_X,vibration_Z,vibration_spindle,noise,flow_rate,cutting_speed,depth_of_cut,diameter,tolerance
count,939284.000000,939284.000000,939284.000000,937240.000000,2044.000000,939284.000000,939284.000000,939284.000000,939284.000000,939284.000000,939284.000000,939284.000000,939284.000000,939284.000000,939284.000000,28991.000000,939284.000000,939284.000000,1.000000e+03
mean,100.729411,1468.452832,493.966563,8.646665,8.582681,49.641929,1685.677974,29.970094,34.035473,0.162494,1.744125,1.709406,2.492975,19.044363,10.765695,134.602497,0.504714,82.499995,1.000000e-01
std,57.794403,288.861147,289.374788,4.656069,4.621081,34.633114,2709.470796,25.579736,12.007477,0.112612,1.739622,1.662902,1.681998,28.243265,11.226119,77.957982,0.285459,2.500001,1.416243e-15
min,1.000000,1000.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80.000000,1.000000e-01
25%,53.000000,1200.000000,242.000000,5.000000,4.000000,22.080511,201.727717,9.181483,25.729915,0.068168,0.441527,0.499517,1.088339,2.411606,0.000000,67.000000,0.257962,80.000000,1.000000e-01
50%,97.000000,1450.000000,494.000000,9.000000,9.000000,44.251131,409.600000,18.398999,34.112778,0.136643,0.884882,1.000000,2.202691,4.832714,0.000000,135.000000,0.515924,80.000000,1.000000e-01
75%,152.000000,1700.000000,738.000000,13.000000,13.000000,68.973020,994.752187,52.073273,43.333333,0.253302,2.906995,2.227431,3.797563,17.948718,20.000000,202.000000,0.742038,85.000000,1.000000e-01
max,199.000000,1950.000000,1000.000000,16.000000,16.000000,120.000000,10000.000000,80.000000,55.000000,0.400000,6.000000,6.000000,6.000000,100.000000,25.000000,269.000000,1.000000,85.000000,1.000000e-01


In [17]:
fig1=plt.figure(figsize=(500,50))
plt.plot(df['spindle_speed'],df['part_count'])


